<a href="https://colab.research.google.com/github/asigalov61/Amazing-Piano/blob/master/Amazing_GPT2_Piano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==1.14
!pip install numpy
!pip install librosa
!pip install tensorboardX
!pip install tqdm
!apt-get install swig
!python3 setup.py install
!pip install git+https://github.com/vishnubob/python-midi@feature/python3
!pip install mido

     |████████████████████████████████| 109.2MB 26kB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 491kB 55.2MB/s 
     |████████████████████████████████| 3.2MB 20.6MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0
     |████████████████████████████████| 317kB 4.8MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Sug

### Data Files Path Preparation

In [2]:
!wget 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'
!unzip -j /content/maestro-v2.0.0-midi.zip

--2020-08-04 04:46:14--  https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59243107 (56M) [application/zip]
Saving to: ‘maestro-v2.0.0-midi.zip’

maestro-v2.0.0-midi 100%[===================>]  56.50M  81.4MB/s    in 0.7s    

2020-08-04 04:46:15 (81.4 MB/s) - ‘maestro-v2.0.0-midi.zip’ saved [59243107/59243107]

Archive:  /content/maestro-v2.0.0-midi.zip
  inflating: maestro-v2.0.0.csv      
  inflating: MIDI-Unprocessed_13_R1_2008_01-04_ORIG_MID--AUDIO_13_R1_2008_wav--1.midi  
  inflating: MIDI-Unprocessed_03_R1_2008_01-04_ORIG_MID--AUDIO_03_R1_2008_wav--1.midi  
  inflating: MIDI-Unprocessed_17_R1_2008_01-04_ORIG_MID--AUDIO_17_R1_2008_wav--4.midi  
  inflating: MIDI-Unprocessed_10_R1_2

In [3]:
import os
from os import listdir
from os.path import isfile, join

# Modify this path to your own MAESTRO dataset
maestro_dir = '/content'

data_dirs = ['/content']
#for year in ['2004', '2006', '2008', '2009', '2011', '2013', '2014', '2015', '2017', '2018']:
#    data_dirs.append(maestro_dir + '/' + str(year))

data_files = []
for data_dir in data_dirs:
    data_files += [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f)) if 'midi' in f]

data_files.sort()

print('total midi files : ', len(data_files))

total midi files :  1283


### Event Extract from Midi File

In [4]:
import mido
import numpy as np


def get_eventlist(data_file):
    ON = 1
    OFF = 0
    CC = 2

    midi = mido.MidiFile(data_file)

    current_time = 0
    eventlist = []
    cc = False
    for msg in midi:
        #print(msg)
        current_time += msg.time

         # NOTE ON CASE
        if msg.type is 'note_on' and msg.velocity > 0:
            event = [current_time, ON, msg.note, msg.velocity]
            eventlist.append(event)

         # NOTE OFF CASE        
        elif msg.type is 'note_off' or (msg.type is 'note_on' and msg.velocity == 0):
            event = [current_time, OFF, msg.note, msg.velocity]
            eventlist.append(event)
            
        if msg.type is 'control_change':
            
            if msg.control != 64:
                continue
            
            if cc == False and msg.value > 0:
                cc = True
                event = [current_time, CC, 0, 1]
                eventlist.append(event)
                
            elif cc == True and msg.value == 0:
                cc = False
                event = [current_time, CC, 0, 0]
                eventlist.append(event)
                
    eventlist = np.array(eventlist)
    return eventlist

index = np.random.randint(0, len(data_files))
print(index)
eventlist = get_eventlist(data_files[index])
print(eventlist)

901
[[  0.           2.           0.           1.        ]
 [  0.97083333   1.          37.          37.        ]
 [  1.06458333   0.          37.           0.        ]
 ...
 [351.38020833   0.          49.           0.        ]
 [351.55416667   0.          56.           0.        ]
 [352.45         2.           0.           0.        ]]


### Midifile to EventListfile

In [ ]:
from tqdm import tqdm_notebook as tqdm
import os

dataset_dir = 'dataset'

if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

for i in tqdm(range(len(data_files))):
    print(data_files[i])
    eventlist = get_eventlist(data_files[i])
    print(eventlist.shape)
    
    save_file = dataset_dir + '/' + str(i)
    data = {'eventlist': eventlist}
    np.savez(save_file, **data, allow_pickle=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


/content/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R1_2014_wav--1.midi
(2914, 4)
/content/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R1_2014_wav--2.midi
(1203, 4)
/content/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R1_2014_wav--3.midi
(19160, 4)
/content/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R1_2014_wav--5.midi
(5675, 4)
/content/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_02_R1_2014_wav--1.midi
(4496, 4)
/content/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_02_R1_2014_wav--2.midi
(17860, 4)
/content/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_02_R1_2014_wav--4.midi
(12610, 4)
/content/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_02_R1_2014_wav--5.midi
(35208, 4)
/content/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_03_R1_2014_wav--2.midi
(2800, 4)
/content/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_03_R1_2014_wav--3.midi
(13610, 4)
/content/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_03_R1_2014_wav--4.midi
(5308, 4)
/content/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_03_R1_2014_wav--5.mi

### Data List

In [ ]:
import os
from os import listdir
from os.path import isfile, join
import ntpath

data_dirs = ['dataset', 'dataset']

data_files = []

for data_dir in data_dirs:
    data_files += [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f)) if '.npz' in f]
print(len(data_files))

data_files.sort()


### Hyperparameters Setting

In [ ]:
IntervalDim = 100

VelocityDim = 32
VelocityOffset = IntervalDim

NoteOnDim = NoteOffDim = 128
NoteOnOffset = IntervalDim + VelocityDim
NoteOffOffset = IntervalDim + VelocityDim + NoteOnDim

CCDim = 2
CCOffset = IntervalDim + VelocityDim + NoteOnDim + NoteOffDim

EventDim = IntervalDim + VelocityDim + NoteOnDim + NoteOffDim + CCDim # 390

Time = 2000

EmbeddingDim = 512

HeadDim = 32
Heads = 16
ContextDim = HeadDim * Heads # 512

Layers = 8

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.training import HParams

def default_hparams():
    return HParams(
        n_vocab=EventDim,
        n_ctx=ContextDim,
        n_embd=EmbeddingDim,
        n_head=Heads,
        n_layer=Layers,
        n_time=Time,
    )

hparams = default_hparams()
print(hparams)

### Load data from npz and converter to token sequence

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa.display

def get_data(length=Time):
    index = np.random.randint(0, len(data_files))
    data = np.load(data_files[index])['eventlist']
    
    # time augmentation
    data[:, 0] *= np.random.uniform(0.80, 1.20)
    
    # absolute time to relative interval
    data[1:, 0] = data[1:, 0] - data[:-1, 0]
    data[0, 0] = 0
    
    # discretize interval into IntervalDim
    data[:, 0] = np.clip(np.round(data[:, 0] * IntervalDim), 0, IntervalDim - 1)
    
    # Note augmentation
    data[:, 2] += np.random.randint(-6, 6)
    data[:, 2] = np.clip(data[:, 2], 0, NoteOnDim - 1)
    
    eventlist = []
    for d in data:
        # append interval
        interval = d[0]
        eventlist.append(interval)
    
        # note on case
        if d[1] == 1:
            velocity = (d[3] / 128) * VelocityDim + VelocityOffset
            note = d[2] + NoteOnOffset
            eventlist.append(velocity)
            eventlist.append(note)
            
        # note off case
        elif d[1] == 0:
            note = d[2] + NoteOffOffset
            eventlist.append(note)
        # CC
        elif d[1] == 2:
            event = CCOffset + d[3]
            eventlist.append(event)
            
    eventlist = np.array(eventlist).astype(np.int)
    
    if len(eventlist) > (length+1):
        start_index = np.random.randint(0, len(eventlist) - (length+1))
        eventlist = eventlist[start_index:start_index+(length+1)]
        
    # pad zeros
    if len(eventlist) < (length+1):
        pad = (length+1) - len(eventlist)
        eventlist = np.pad(eventlist, (pad, 0), 'constant')
        
    x = eventlist[:length]
    y = eventlist[1:length+1]
    
    return x, y
    
x, y = get_data()
print('x shape : ', x.shape)
print('y shape : ', y.shape)
# print(x)
# print(y)
    
    
roll = np.zeros([len(x), EventDim])
for t, _x in enumerate(x):
    roll[t, _x] = 1

plt.figure(figsize=[18, 15])
librosa.display.specshow(roll.T)
plt.show()

### GPT-2 source code from https://github.com/openai/gpt-2/blob/master/src/model.py

In [ ]:
def shape_list(x):
    """Deal with dynamic shape in tensorflow cleanly."""
    static = x.shape.as_list()
    dynamic = tf.shape(x)
    return [dynamic[i] if s is None else s for i, s in enumerate(static)]

def softmax(x, axis=-1):
    x = x - tf.reduce_max(x, axis=axis, keepdims=True)
    ex = tf.exp(x)
    return ex / tf.reduce_sum(ex, axis=axis, keepdims=True)

def gelu(x):
    return 0.5*x*(1+tf.tanh(np.sqrt(2/np.pi)*(x+0.044715*tf.pow(x, 3))))

def norm(x, scope, *, axis=-1, epsilon=1e-5):
    """Normalize to mean = 0, std = 1, then do a diagonal affine transform."""
    with tf.variable_scope(scope):
        n_state = x.shape[-1].value
        g = tf.get_variable('g', [n_state], initializer=tf.constant_initializer(1))
        b = tf.get_variable('b', [n_state], initializer=tf.constant_initializer(0))
        u = tf.reduce_mean(x, axis=axis, keepdims=True)
        s = tf.reduce_mean(tf.square(x-u), axis=axis, keepdims=True)
        x = (x - u) * tf.rsqrt(s + epsilon)
        x = x*g + b
        return x

def split_states(x, n):
    """Reshape the last dimension of x into [n, x.shape[-1]/n]."""
    *start, m = shape_list(x)
    return tf.reshape(x, start + [n, m//n])

def merge_states(x):
    """Smash the last two dimensions of x into a single dimension."""
    *start, a, b = shape_list(x)
    return tf.reshape(x, start + [a*b])

def conv1d(x, scope, nf, *, w_init_stdev=0.02):
    with tf.variable_scope(scope):
        *start, nx = shape_list(x)
        w = tf.get_variable('w', [1, nx, nf], initializer=tf.random_normal_initializer(stddev=w_init_stdev))
        b = tf.get_variable('b', [nf], initializer=tf.constant_initializer(0))
        c = tf.reshape(tf.matmul(tf.reshape(x, [-1, nx]), tf.reshape(w, [-1, nf]))+b, start+[nf])
        return c

def attention_mask(nd, ns, *, dtype):
    """1's in the lower triangle, counting from the lower right corner.
    Same as tf.matrix_band_part(tf.ones([nd, ns]), -1, ns-nd), but doesn't produce garbage on TPUs.
    """
    i = tf.range(nd)[:,None]
    j = tf.range(ns)
    m = i >= j - ns + nd
    return tf.cast(m, dtype)

'''
MEMORY EFFICIENT IMPLEMENTATION OF RELATIVE POSITION-BASED ATTENTION
(Music Transformer, Cheng-Zhi Anna Huang et al. 2018)
'''
def attn(x, scope, n_state, *, hparams):
    assert x.shape.ndims == 3  # Should be [batch, sequence, features]
    assert n_state % hparams.n_head == 0

    def split_heads(x):
        # From [batch, sequence, features] to [batch, heads, sequence, features]
        return tf.transpose(split_states(x, hparams.n_head), [0, 2, 1, 3])

    def merge_heads(x):
        # Reverse of split_heads
        return merge_states(tf.transpose(x, [0, 2, 1, 3]))

    def mask_attn_weights(w):
        # w has shape [batch, heads, dst_sequence, src_sequence], where information flows from src to dst.
        _, _, nd, ns = shape_list(w)
        b = attention_mask(nd, ns, dtype=w.dtype)
        b = tf.reshape(b, [1, 1, nd, ns])
        w = w*b - tf.cast(1e10, w.dtype)*(1-b)
        return w
    
    def relative_attn(q):
        # q have shape [batch, heads, sequence, features]
        batch, heads, sequence, features = shape_list(q)
        E = tf.get_variable('E', [heads, sequence, features])
        # [heads, batch, sequence, features]
        q_ = tf.transpose(q, [1, 0, 2, 3])
        # [heads, batch * sequence, features]
        q_ = tf.reshape(q_, [heads, batch * sequence, features])
        # [heads, batch * sequence, sequence]
        rel = tf.matmul(q_, E, transpose_b=True)
        # [heads, batch, sequence, sequence]
        rel = tf.reshape(rel, [heads, batch, sequence, sequence])
        # [heads, batch, sequence, 1+sequence]
        rel = tf.pad(rel, ((0, 0), (0, 0), (0, 0), (1, 0)))
        # [heads, batch, sequence+1, sequence]
        rel = tf.reshape(rel, (heads, batch, sequence+1, sequence))
        # [heads, batch, sequence, sequence]
        rel = rel[:, :, 1:]
        # [batch, heads, sequence, sequence]
        rel = tf.transpose(rel, [1, 0, 2, 3])
        return rel
        
    def multihead_attn(q, k, v):
        # q, k, v have shape [batch, heads, sequence, features]
        w = tf.matmul(q, k, transpose_b=True)
        w = w + relative_attn(q)
        w = w * tf.rsqrt(tf.cast(v.shape[-1].value, w.dtype))

        w = mask_attn_weights(w)
        w = softmax(w)
        a = tf.matmul(w, v)
        return a

    with tf.variable_scope(scope):
        c = conv1d(x, 'c_attn', n_state*3)
        q, k, v = map(split_heads, tf.split(c, 3, axis=2))
        present = tf.stack([k, v], axis=1)

        a = multihead_attn(q, k, v)
        a = merge_heads(a)
        a = conv1d(a, 'c_proj', n_state)
        return a, present


def mlp(x, scope, n_state, *, hparams):
    with tf.variable_scope(scope):
        nx = x.shape[-1].value
        h = gelu(conv1d(x, 'c_fc', n_state))
        h2 = conv1d(h, 'c_proj', nx)
        return h2


def block(x, scope, *, hparams):
    with tf.variable_scope(scope):
        nx = x.shape[-1].value
        a, present = attn(norm(x, 'ln_1'), 'attn', nx, hparams=hparams)
        x = x + a
        m = mlp(norm(x, 'ln_2'), 'mlp', nx*4, hparams=hparams)
        x = x + m
        return x, present

def expand_tile(value, size):
    """Add a new axis of given size."""
    value = tf.convert_to_tensor(value, name='value')
    ndims = value.shape.ndims
    return tf.tile(tf.expand_dims(value, axis=0), [size] + [1]*ndims)

def model(hparams, X, scope='model', reuse=False):
    with tf.variable_scope(scope, reuse=reuse):
        results = {}
        batch, sequence = shape_list(X)

        wte = tf.get_variable('wte', [hparams.n_vocab, hparams.n_embd],
                             initializer=tf.random_normal_initializer(stddev=0.02))
        h = tf.gather(wte, X)

        # Transformer
        presents = []
        for layer in range(hparams.n_layer):
            h, present = block(h, 'h%d' % layer, hparams=hparams)
            presents.append(present)
        results['present'] = tf.stack(presents, axis=1)
        h = norm(h, 'ln_f')

        # Language model loss.  Do tokens <n predict token n?
        h_flat = tf.reshape(h, [batch*sequence, hparams.n_embd])
        logits = tf.matmul(h_flat, wte, transpose_b=True)
        logits = tf.reshape(logits, [batch, sequence, hparams.n_vocab])
        results['logits'] = logits
        return results

### Draw Main Graph

In [ ]:
hparams = default_hparams()
print(hparams)


tf.reset_default_graph()

X = tf.placeholder(tf.int32, [None, hparams.n_time])
Y = tf.placeholder(tf.int32, [None, hparams.n_time])

X_onehot = tf.one_hot(X, axis=2, depth=hparams.n_vocab)

logits = model(hparams, X)['logits']
probs = tf.nn.softmax(logits, axis=2)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits)
loss = tf.reduce_mean(cross_entropy)

#temperature = tf.Variable(1., name='temperature')
temperature = 0
u = tf.random.uniform(shape=tf.shape(logits[:, -1]), minval=1e-5, maxval=1.-1e-5)
u = (logits[:, -1] - tf.log(temperature + 1e-8)) - tf.log(-tf.log(u))
sample = tf.argmax(u, axis=1)

#dist = tf.distributions.Categorical(logits=logits[:, -1])
#sample = dist.sample()

'''
Train
'''
global_step = tf.Variable(0, name='global_step')
learning_rate = tf.Variable(1e-3, name='learning_rate')
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step)

'''
Session Open
'''


# GPU number to use
gpu_options = tf.GPUOptions(visible_device_list="0")
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

# config = tf.ConfigProto(device_count = {'GPU': 0})
# sess = tf.Session(config=config)

sess.run(tf.global_variables_initializer())

print('graph create')

### Load model if exist

In [ ]:
import tensorflow.contrib.slim as slim
from tensorflow.python import pywrap_tensorflow

load_dir = 'save/gpt2-cc-interval100-attention2000-midi'
save_dir = 'save/gpt2-cc-interval100-attention2000-midi'

def get_variables_from_checkpoint_file(file_name):
    variables = []
    reader = pywrap_tensorflow.NewCheckpointReader(file_name)

    var_to_shape_map = reader.get_variable_to_shape_map()
    for key in sorted(var_to_shape_map):
        variables.append((key, var_to_shape_map[key]))

    return variables

saver = tf.train.Saver()

if True:
    restore_file = tf.train.latest_checkpoint(load_dir)
    if restore_file is not None:
        try:
            saver.restore(sess, restore_file)
            print("Model restored.", restore_file)
        except:
            saved_variables = get_variables_from_checkpoint_file(restore_file)
            model_variables = slim.get_variables_to_restore()
            restore_variables = []
            for model_variable in model_variables:
                for saved_variable_name, saved_variable_shape in saved_variables:
                    model_variable_name = model_variable.name.split(":")[0]
                    if saved_variable_name == model_variable_name and tuple(saved_variable_shape) == model_variable.shape:
                        restore_variables.append(model_variable)

            init_saver = tf.train.Saver(restore_variables)
            init_saver.restore(sess, restore_file)
            print("Model partially restored.")
    else:
        print('model not exist.')
        

### TensorboardX Logger

In [ ]:
from tensorboardX import SummaryWriter

class Logger(SummaryWriter):
    def __init__(self, logdir):
        super(Logger, self).__init__(logdir)

    def log(self, log_string, value, iteration):
            self.add_scalar(log_string, value, iteration)
            
logger = Logger(save_dir)            


### Train Loop

In [ ]:
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import librosa.display
from time import sleep
import time

batch_size = 1

while(True):
    for _ in range(100):
        _inputs = []
        _targets = []
        for _ in range(batch_size):
            while(True):
                x, y = get_data(hparams.n_time)
                if(x.shape == y.shape):
                    break
                 
            _inputs.append(x)
            _targets.append(y)
        _inputs = np.stack(_inputs)
        _targets = np.stack(_targets)
        print(_inputs.shape, _targets.shape)
        
        _, _global_step, _loss = sess.run([train_step, global_step, loss], 
                                          feed_dict={X: _inputs, 
                                                     Y: _targets,
                                                     learning_rate: 1e-3})
        print(_global_step, _loss)
        
        if _global_step % 10 == 0:
            logger.log('loss', _loss, _global_step)
        
        if _global_step % 1000 == 0:
            save_path = saver.save(sess, save_dir + '/checkpoint', global_step=_global_step)
            print("Model saved in path: %s" % save_path)
        
    clear_output()
    
    _inputs_onehot, _probs = sess.run([X_onehot, probs], feed_dict={X: _inputs})
    
    plt.figure(figsize=[18, 8])
    librosa.display.specshow(_inputs_onehot[0].T)
    plt.show()
    
    plt.figure(figsize=[18, 8])
    librosa.display.specshow(_probs[0].T)
    plt.show()